In [ ]:
!pip install librosa numpy pandas scikit-learn tensorflow matplotlib

In [ ]:
pip list


In [ ]:
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt


In [ ]:
import os

In [ ]:
train_dir = './track-2-audio-identification-quest/training_split/training_split'
test_dir = './track-2-audio-identification-quest/testing_split/testing_split'

In [ ]:
def get_list_of_files(dir_name):
    list_of_files = os.listdir(dir_name)
    all_files = []
    for entry in list_of_files:
        full_path = os.path.join(dir_name, entry)
        if os.path.isdir(full_path):
            all_files.extend(get_list_of_files(full_path))
        else:
            all_files.append(full_path)
    return all_files

In [ ]:
# Get list of files
train_files = get_list_of_files(train_dir)
test_files = get_list_of_files(test_dir)

print(f"Number of training files: {len(train_files)}")
print(f"Number of testing files: {len(test_files)}")

In [ ]:
# Load a sample audio file
sample_file = train_files[0]  # Just picking the first training file for now
audio, sr = librosa.load(sample_file, sr=None)  # sr=None to keep the original sample rate

print(f"Audio sample shape: {audio.shape}")
print(f"Sample rate: {sr}")

In [ ]:
def extract_mfcc(file_path, n_mfcc=20):
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfccs, axis=1)  # Averaging over time

# Extract MFCC features for a sample file
mfcc_features = extract_mfcc(sample_file)
print(f"MFCC features shape: {mfcc_features.shape}")

In [ ]:
def extract_features_from_files(file_paths, feature_extractor):
    features = []
    for file_path in file_paths:
        feature = feature_extractor(file_path)
        features.append(feature)
    return np.array(features)

# Extract features for training and testing files
X_train = extract_features_from_files(train_files, extract_mfcc)
X_test = extract_features_from_files(test_files, extract_mfcc)

print(f"Training features shape: {X_train.shape}")
print(f"Testing features shape: {X_test.shape}")


In [ ]:
import re

def extract_labels_from_filename(filename):
    # Example filename: '132_70_female.wav'
    print(f"Processing filename:{filename}")
    match = re.search(r'(\d+)_([\d]+)_(male|female)_(\d+)\.wav', filename)
    if match:
        age = int(match.group(2))
        gender = match.group(3)
        
        # Determine age label
        if age <= 15:
            age_label = 0
        elif 16 <= age <= 40:
            age_label = 1
        else:
            age_label = 2
        
        # Determine gender label
        gender_label = 0 if gender == 'male' else 1
        
        return age_label, gender_label
    else:
        raise ValueError("Filename does not match expected format")


In [ ]:
def extract_labels_from_files(file_paths):
    age_labels = []
    gender_labels = []
    for file_path in file_paths:
        try:
            age_label, gender_label = extract_labels_from_filename(os.path.basename(file_path))
            age_labels.append(age_label)
            gender_labels.append(gender_label)
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")
            
    return np.array(age_labels), np.array(gender_labels)


In [ ]:
y_train_age, y_train_gender = extract_labels_from_files(train_files)

print(f"Training age labels shape: {y_train_age.shape}")
print(f"Training gender labels shape: {y_train_gender.shape}")

from sklearn.model_selection import train_test_split

# Split training data into training and validation sets
X_train, X_val, y_train_age, y_val_age, y_train_gender, y_val_gender = train_test_split(
    X_train, y_train_age, y_train_gender, test_size=0.2, random_state=42
)
#meaning age labels that i have made classified now
print(f"Validation features shape: {X_val.shape}")
print(f"Validation age labels shape: {y_val_age.shape}")
print(f"Validation gender labels shape: {y_val_gender.shape}")


In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and validation data
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Input
from tensorflow.keras.optimizers import Adam

# Define the model
input_layer = Input(shape=(X_train.shape[1], 1))
x = Conv1D(32, 3, activation='relu')(input_layer)
x = MaxPooling1D(2)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)

# Define two separate output layers
age_output = Dense(3, activation='softmax', name='age_output')(x)
gender_output = Dense(2, activation='softmax', name='gender_output')(x)

# Create the model
model = Model(inputs=input_layer, outputs=[age_output, gender_output])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss={'age_output': 'sparse_categorical_crossentropy', 'gender_output': 'sparse_categorical_crossentropy'},
    metrics={'age_output': 'accuracy', 'gender_output': 'accuracy'}
)


In [ ]:
model.summary()

In [ ]:
#reshaping
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print(X_train.shape)  
print(X_val.shape)    



In [ ]:
history = model.fit(
    X_train, 
    {'age_output': y_train_age, 'gender_output': y_train_gender},
    epochs=200,  
    batch_size=32,  
    validation_data=(X_val, {'age_output': y_val_age, 'gender_output': y_val_gender}),
    verbose=1  # Set to 1 to see progress during training
)


In [ ]:
# Extract features for test files
X_test = extract_features_from_files(test_files, extract_mfcc)
X_test = scaler.transform(X_test)  # Apply the same scaling as training data
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))  # Reshape for the model


In [ ]:
print(f"Number of test files: {len(test_files)}")
print(f"Shape of X_test: {X_test.shape}")

In [ ]:
print(f"Shape of X_test: {X_test.shape}")

In [ ]:
predictions = model.predict(X_test, batch_size=32)   #637/32

In [ ]:
# Define thresholds for fallback logic
lower_bound_threshold = 0.8 # Lower bound for confidence

# Extract probabilities for each sample
predicted_age_probs = predictions[0]  # Probabilities for age groups
predicted_gender_probs = predictions[1]
# Initialize an array for predicted labels
predicted_age_labels = np.argmax(predicted_age_probs, axis=1)
predicted_gender_labels = np.argmax(predicted_gender_probs, axis=1)

# Apply fallback mechanism so 0 can be included now,
for i, probs in enumerate(predicted_age_probs):
    if np.max(probs) < lower_bound_threshold:
        predicted_age_labels[i] = 0  # Assign to age group 0 if below threshold


# Print some predictions to check
print(predicted_age_labels[:10])  # Print first 10 predictions
print(predicted_gender_labels[:10])




In [ ]:
import pandas as pd
import os

# Redefine the DataFrame with .wav extension removed
df = pd.DataFrame({
    'File_name': [os.path.splitext(os.path.basename(file))[0] for file in test_files],
    'Age_group': predicted_age_labels,
    'Gender': ['Male' if gender == 0 else 'Female' for gender in predicted_gender_labels]
})

print(f"Number of rows in DataFrame: {len(df)}")

# Print the first few rows to check the DataFrame
print(df.head())


In [1]:
output_csv_path = 'new.csv'
df.to_csv(output_csv_path, index=False)

#finished !!

NameError: name 'df' is not defined

In [ ]:
#not req really from now just for testing...
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# After training the model and making predictions
y_val_pred = model.predict(X_val)
y_val_age_pred = np.argmax(y_val_pred[0], axis=1)
y_val_gender_pred = np.argmax(y_val_pred[1], axis=1)

# Confusion matrix for age and gender
age_cm = confusion_matrix(y_val_age, y_val_age_pred)
gender_cm = confusion_matrix(y_val_gender, y_val_gender_pred)

# Print confusion matrices
print("Age Confusion Matrix:")
print(age_cm)
print("Gender Confusion Matrix:")
print(gender_cm)

# Visualize the confusion matrix for age
plt.figure(figsize=(8, 6))
sns.heatmap(age_cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['0-15', '16-40', '41+'], 
            yticklabels=['0-15', '16-40', '41+'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Age Confusion Matrix')
plt.show()

# Visualize the confusion matrix for gender
plt.figure(figsize=(8, 6))
sns.heatmap(gender_cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Male', 'Female'], 
            yticklabels=['Male', 'Female'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Gender Confusion Matrix')
plt.show()


In [ ]:
print(predicted_age_probs[:10])  # Print probabilities for the first 10 samples

In [ ]:
# Create a list of test file names
test_file_names = [os.path.basename(file) for file in test_files]

# Define the index you want to check
sample_index = 93# For example, if the index is 1

# Get the filename for that index
filename = test_file_names[sample_index]

# Print the filename
print(f"The filename for index {sample_index} is: {filename}")



In [ ]:

specific_age_probs = predicted_age_probs[sample_index]
specific_gender_probs = predicted_gender_probs[sample_index]

print(f"Probabilities for age for sample {sample_index}: {specific_age_probs}")
print(f"Probabilities for gender for sample {sample_index}: {specific_gender_probs}")
